# 75.06 Organización de datos: Trabajo Práctico 2
Integrantes del grupo
- Avecilla, Ignacio - 105067
- Balmaceda, Fernando - 105525
- Singer, Joaquín - 105854
- Villegas, Tomás - 106456

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

ds_path = None
pd.set_option('display.max_columns', None)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if "Womens" in filename:
            ds_path = dirname+"/"+filename

# Parte 1

# Analisis exploratorio

In [ ]:
df = pd.read_csv(ds_path)

In [ ]:
df.head()

In [ ]:
df.info()

## Descripcion de las columnas:

1. Clothing ID: id del producto en especifico que esta siendo reseñado.
2. Age: edad de los reseñadores.
3. Title: titulo de la review.
4. Review Text: texto de la review.
5. Rating: calificacion del reseñador (del 1 al 5).
6. Recommended IND: variable binaria donde 1 es recomendado, 0 es no recomendado.
7. Positive Feedback Count: cantidad de reseñadores que tienen una calificacion positiva sobre el producto.
8. Division Name: categoria de la division del producto.
9. Department Name: categoria del departamento del producto.
10. Class Name: categoria de la clase de producto.

# Ingenieria de Features

In [ ]:
df.isna().sum() / df.shape[0] * 100

Se ve que la columna con mas porcentaje de NaNs es "Title". Pero como es una columna sobre la que no se va a trabajar podemos no tenerla en cuenta. Lo mismo para "Division Name", "Department Name" y "Class Name".

Sin embargo, la columna "Review Text" tiene que estar limpia de NaNs por ser la variable que justamente vamos a analizar, por ende se eliminaran las filas que tengan NaN en esa columna.

In [ ]:
df.dropna(subset=["Review Text"], inplace=True)

In [ ]:
df.isna().sum() / df.shape[0] * 100

Se agrega una columna binaria "Objective" con "Positive" y "Negative" segun la columna "Rating"

Ademas, se agrega una columna que identifique por numero a "Positive" -> 1 y "Negative" -> 0, para utilizarla en los modelos de prediccion.

In [ ]:
df["Objective"] = df["Rating"].map(lambda x: "Negative" if x<=3 else "Positive")
df["Binary Objective"] = df["Objective"].map(lambda x: 0 if x=="Negative" else 1)
df.shape

Se eliminan filas que tengan "Recommended IND"=1 tiene "Binary Objective"=0 y viceversa, ya que consideramos que es algun error en la carga de los datos que carecen de sentido y puede ser controversial para nuestro entrenamiento

In [ ]:
df.drop(df[df["Binary Objective"] != df["Recommended IND"]].index, inplace=True)

# Pre-procesamiento de modelos

A este punto consideramos que el dataframe esta lo suficientemente limpio como para iniciar el proceso de entrenamiento de modelos.

Procedemos a dividir el dataframe en train (70%) y test (30%)

In [ ]:
df_train, df_test = train_test_split(df, test_size = 0.3, random_state = 42)

El CountVectorizer transforma el texto en un vector basado en la frequencia de cada palabra en cada texto. De esta forma obtenemos una matriz con los vectores de cada texto, el cual reciben los modelos. Realizamos esto porque los modelos no interpretan texto pero si numeros.

In [ ]:
cv = CountVectorizer(binary=True)
cv.fit(df['Review Text'].values)

In [ ]:
x_train = cv.transform(df_train["Review Text"].values)
x_test = cv.transform(df_test["Review Text"].values)

y_train = df_train["Binary Objective"].values
y_test = df_test["Binary Objective"].values

## Modelo de Regresion Logística

Entrenamos el modelo de regresion logistica

In [ ]:
modelo_reglog = LogisticRegression(solver = 'liblinear', random_state = 42)
modelo_reglog.fit(x_train,y_train)

Evaluamos el modelo con las diferentes metricas pedidas comparandolo con el split de test que hicimos anteriormente

In [ ]:
y_pred = modelo_reglog.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC score:", round(metrics.roc_auc_score(y_test, y_pred), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

El modelo de regresion logistica se comporta muy bien a la hora de clasificar los casos positivos ya que tenemos muchas mas entradas con esta caracterisitica pero aun asi se comporta bien tambien con las clasificaciones negativas.

## Modelo de Random Forest

Entrenamos el modelo de Random Forest

In [ ]:
modelo_ranfo = RandomForestClassifier()
modelo_ranfo.fit(x_train, y_train)

Evaluamos el modelo con las diferentes metricas pedidas comparandolo con el split de test que hicimos anteriormente

In [ ]:
y_pred = modelo_ranfo.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC score:", round(metrics.roc_auc_score(y_test, y_pred), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

Como podemos ver este modelo es muy bueno para predecir las reviews positivas, de hecho casi todas estan bien clasificadas. Sin embargo clasifica muy mal los casos negativos. Es probable que esto se de por el desbalanceo que esta presente en el dataset

## Modelo de Naive Bayes

Entrenamos el modelo de Naive Bayes, utilizamos multinomial Naive Bayes ya que es el apropiado para entrenar con textos

In [ ]:
modelo_naba = MultinomialNB()
modelo_naba.fit(x_train, y_train)

Evaluamos el modelo con las diferentes metricas pedidas comparandolo con el split de test que hicimos anteriormente

In [ ]:
y_pred = modelo_naba.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC score:", round(metrics.roc_auc_score(y_test, y_pred), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

El modelo de Naive Bayes es bastante bueno con ambos valores del target. Clasifica la mayoria de los resultados correctamente y las metricas son de las mejores por el momento

## Modelo de Arbol de Decisiones

Entrenamos el modelo de Arbol de decisiones

In [ ]:
modelo_arde = DecisionTreeClassifier(random_state=42)
modelo_arde.fit(x_train, y_train)

Evaluamos el modelo con las diferentes metricas pedidas comparandolo con el split de test que hicimos anteriormente

In [ ]:
y_pred = modelo_arde.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC score:", round(metrics.roc_auc_score(y_test, y_pred), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

El modelo de arboles decisiones mejora un poco respecto de random forest ya que clasifica los casos negativos casi como si fuera random, pero sigue siendo peor que naive bayes y regresion logistica

# Utilizando UnderSampling

Nos dimos cuenta que el dataset estaba muy desbalanceado en cuanto a la cantidad de reviews positivas y negativas que teniamos asi que optamos por probar hacer undersampling con los datos para igualar la cantidad de reviews y ver si los modelos mejoraban al aplicar esta tecnica

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
x_train, y_train = rus.fit_resample(x_train, y_train)

## Modelo de Regresion Logistica

In [ ]:
modelo_reglog2 = LogisticRegression(solver = 'liblinear', random_state = 42)
modelo_reglog2.fit(x_train,y_train)

In [ ]:
y_pred = modelo_reglog2.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC score:", round(metrics.roc_auc_score(y_test, y_pred), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

## Modelo Random Forest

In [ ]:
modelo_ranfo2 = RandomForestClassifier()
modelo_ranfo2.fit(x_train, y_train)

In [ ]:
y_pred = modelo_ranfo2.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC score:", round(metrics.roc_auc_score(y_test, y_pred), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

## Modelo Naive Bayes

In [ ]:
modelo_naba2 = MultinomialNB()
modelo_naba2.fit(x_train, y_train)

In [ ]:
y_pred = modelo_naba2.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC score:", round(metrics.roc_auc_score(y_test, y_pred), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

## Modelo Arbol de Decisiones

In [ ]:
modelo_arde2 = DecisionTreeClassifier(random_state=42)
modelo_arde2.fit(x_train, y_train)

In [ ]:
y_pred = modelo_arde2.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC score:", round(metrics.roc_auc_score(y_test, y_pred), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

Vemos que gracias al UnderSampling hubo una mejora en el desempeño de los modelos, dado que no los estamos condicionando a solo predecir el valor 1 sino que estos datos estan balanceados. Esto nos sirve para ver la utilizacion de esta tecnica y como impactaba en un ejemplo practico, pero para las conclusiones de nuestro analisis vamos a analizar los resultados con el dataset original tal como lo pide la consigna

### Mejor modelo: Naive Bayes

Dado que el modelo de Naive Bayes es el que mejor desempeño tuvo (ROC-AUC score: 0.8375) respecto de los demás modelos, lo vamos a utilizar para predecir la columna "Rating" respecto de "Review Text".

Como el modelo anterior esta entrenado para categorizar de forma binaria (en este caso: 0,1), creamos una nueva instancia de Naive Bates que pueda categorizar más de dos clases (en este caso: 1,2,3,4,5).

In [ ]:
# realizamos denuevo el split dado que al conjunto de prueba anterior se le realizo undersampling

df_train, df_test = train_test_split(df, test_size = 0.3, random_state = 42)
x_train = cv.transform(df_train['Review Text'].values)
y_train = df_train["Rating"].values
x_test = cv.transform(df_test["Review Text"].values)
y_test = df_test["Rating"].values

In [ ]:
modelo = MultinomialNB()
modelo.fit(x_train, y_train)

In [ ]:
y_pred = modelo.predict(x_test)
print(classification_report(y_test, y_pred))

## Conclusiones

Podemos observar que el modelo de Naive Bayes, que obtuvo muy buenos resultados en comparación a otros modelos en el caso binario, no mantuvo el mismo desempeño utilizándolo con las cinco clases, tanto la precisión como el recall caen abismalmente. De todas formas, para completar el experimento faltaría corroborar que efectivamente estos resultados son mejores que con los otros métodos para este caso. También sería interesante ver qué ocurre aplicando undersampling.

Sobre los diferentes métodos pudimos aprender que, para este caso de análisis, los modelos de regresión logística y Naive Bayes consiguieron resultados superiores a los de 
árbol de decisiones y random forest. Aunque con undersampling pudimos aumentar significativamente el recall de random forest sin perder demasiada precisión, haciendo que este último obtenga resultados similares a los de los otros dos métodos.



# Parte 2

# Analisis exploratorio

In [ ]:
url = "https://drive.google.com/uc?export=download&id=1Byz6xmeu_L8e_pPJdlbWA0JsfhMI9-fr"
df_hoteles = pd.read_csv(url)

In [ ]:
df_hoteles.head()

In [ ]:
df_hoteles.info()

| Variable      | descripcion | tipo |
| ----------- | ----------- | ----------- |
| hotel       | nombre del hotel en el cual se hizo la reserva| categorica |
| is_canceled       | Valor que indica si la reserva fue cancelada (1) o no (0)| categorica |
| lead_time       | Diferencia de dias entre la fecha que se hizo la reserva y la fecha en que se llego al establecimiento | numerica |
| arrival_date_year       | Año de llegada al destino | numerica |
| arrival_date_month       | Mes de llegada al destino| categorica |
| arrival_date_week_number      | Numero de semana de llegada al destino| numerica |
| arrival_date_day_of_month       | Dia del mes de llegada al destino| numerica |
| stays_in_weekend_nights  | Cantidad de noches del fin de semana que incluye la reserva | numerica |
| stays_in_week_nights       | Cantidad de noches de semana que incluye la reserva| numerica |
| adults       | Cantidad de adultos | numerica |
| children       | Cantidad de niños | numerica |
| meal       | Tipo de plan de comidas que se eligio en la reserva: Undefined/SC (Sin plan de comidas), BB (Bed & Breakfast), HB (Half board) o FB (Full board) | categorica |
| country       | Pais de origen representadas por sus iniciales | categorica |
| market_segment       | Segmento del mercado designado: TA (Travel Agents) o TO (Tour Operators) | categorica |
| distribution_channel       | Medio por donde se hizo la reserva: TA (Travel Agents) o TO (Tour Operators) | categorica |
| is_repeated_guest       | Valor que representa si una persona ya habia reservado antes (1) o no (0) | categorica |
| previous_cancellation       | Cantidad de veces que la persona cancelo reservas. En caso de que no se encuentren reservas previas de la persona entonces el valor es 0 | numerica |
| previous_bookings_not_canceled      | Cantidad de veces que la persona hizo una reserva y no la cancelo. En caso de que no se encuentren reservas previas de la persona entonces el valor es 0 | numerica |
| reserved_room_type       | Codigo de tipo de habitacion de la reserva | categorica |
| assigned_room_type      | Codigo de tipo de habitacion que fue asignado a la reserva. Puede diferir del reservado por temas administrativos | categorica |
| booking_changes       | Cantidad de veces que se cambio cualquier aspecto relacionado a la reserva | numerica |
| deposit_type       | Valor que representa si se dejo una seña de la reserva y de que tipo | categorica |
| agent       | ID de la agencia de viajes con la que se reservo | categorica |
| company      | ID de la compañia con la que se reservo  | categorica |
| days_in_waiting_list       | Cantidad de dias que la reserva estuvo en lista de espera antes de ser confirmada | numerica |
| customer_type     | Tipo de reserva | categorica |
| adr       | Tarifa media diaria. Se calcula dividiendo la suma de todas las transacciones de alojamiento por el número total de noches de estancia. | numerica |
| required_car_parking_spaces       | Cantidad de lugares de estacionamiento requeridos por la persona | numerica |
| total_of_special_requests      | Cantidad de pedidos especiales o particulares hechas por la persona | numerica |
| reservation_status       | Ultimo estado registrado para una reserva | categorica |
| reservation_status_date       | Fecha de la ultima actualizacion de estado que tuvo la reserva | numerica |

In [ ]:
def make_plot_for_categorical(category):
    plt.figure(figsize = (10,8))
    sns.set(font_scale = 1.5)
    ax = sns.countplot(y=category, data=df_hoteles, color="skyblue", order = df_hoteles[category].value_counts().iloc[:15].index, )
    ax.set_xlabel("Frequency", fontsize = 20)
    _ = ax.set_ylabel((" ".join(category.split("_"))).capitalize(), fontsize = 20)

Creamos diferentes graficos de barras para las variables categoricas presentes en el dataset, en este caso limitamos la cantidad de resultados a los 15 mas frecuentes para que sea facil de visualizar. Las variables que superan mayor cantidad de valores son "country", "company" y "agent" pero la frecuencia es muy baja a partir del decimo valor asi que decidimos cortarla en el grafico.

In [ ]:
make_plot_for_categorical("hotel")
make_plot_for_categorical("is_canceled")
make_plot_for_categorical("arrival_date_month")
make_plot_for_categorical("meal")
make_plot_for_categorical("country")
make_plot_for_categorical("market_segment")
make_plot_for_categorical("distribution_channel")
make_plot_for_categorical("is_repeated_guest")
make_plot_for_categorical("reserved_room_type")
make_plot_for_categorical("assigned_room_type")
make_plot_for_categorical("deposit_type")
make_plot_for_categorical("customer_type")
make_plot_for_categorical("reservation_status")
make_plot_for_categorical("company")
make_plot_for_categorical("agent")

## Ingenieria de Features

In [ ]:
df_hoteles.isna().sum() / df_hoteles.shape[0] * 100

Como las descripciones de las variables nos lo dicen, las variables "Agent" y "Company" tienen un valor especifico que puede se NaN que representa a una categoria mas, la categoria seria que no hay agencia o compañia involucrada en la reserva. Entonces podemos reemplazar los valores nulos por un nuevo valor 0 que indique que en verdad no existe una agencia o compania involucrada en esa reserva

In [ ]:
df_hoteles['agent'].replace(to_replace=np.nan, value=0, inplace=True)
df_hoteles['company'].replace(to_replace=np.nan, value=0, inplace=True)

Por otro lado vemos que hay 4 valores faltantes en la variable "children", la descripcion de la variable no nos brinda ningun caso particular para que esa variable tenga valores nulos. Como unicamente representan 4 valores entre los 119390 valores totales, podemos eliminar esas observaciones sin riesgo de perder muchos datos valiosos, ya que representarian menos que el 1% del total de datos que tenemos

In [ ]:
df_hoteles.dropna(subset=['children'], inplace=True)

Por ultimo vemos que tenemos valores faltantes en el feature "country" pero la descripcion de las variables dicen que esto se debe a que es posible que no se sepa el pais de origen de la persona en algunas reservas entonces ese campo puede no llenarse. Para esto vamos a incluirlo como una nueva categoria que sea "Desconocido" asi podemos tenerla en cuenta para nuestros analisis y ver si es util.

In [ ]:
df_hoteles['country'].fillna("Desconocido", inplace=True)

----

Vamos a ver, en porcentaje, las variables mas correlacionadas con nuestra variable target que es si la reserva se cancelo o no

In [ ]:
df_hoteles.corr()[['is_canceled']].sort_values('is_canceled', ascending=False) * 100

### Variables mas correlacionadas
- lead_time
- total_of_special_requests
- required_car_parking_spaces
- booking_changes
- previous_cancellations

Ahora realizamos simplemente un scatter plot siendo el eje x la variable correlacionada y el eje y la cancelacion

In [ ]:
sns.scatterplot(data=df_hoteles, x="lead_time", y="is_canceled")
plt.show()

In [ ]:
sns.scatterplot(data=df_hoteles, x="previous_cancellations", y="is_canceled")
plt.show()

In [ ]:
sns.scatterplot(data=df_hoteles, x="total_of_special_requests", y="is_canceled")
plt.show()

In [ ]:
sns.scatterplot(data=df_hoteles, x="required_car_parking_spaces", y="is_canceled")
plt.show()

In [ ]:
sns.scatterplot(data=df_hoteles, x="booking_changes", y="is_canceled")
plt.show()

# Pre-procesamiento para modelo de prediccion

Para empezar con nuestro modelo de prediccion decidimos sacar las siguientes columnas:

- `reservation_status` y `is_canceled` son variables que contienen el valor target y no queremos tener filtrado de datos en nuestro modelo para poder entrenarlo asi que decidimos quitarlas para entrenar.
- La variable `country` tiene muchisimos valores para poder ser encodeado y nuestros datos crecen demasiado en cuanto a dimensionalidad asi que decidimos no tomarla, no creemos que sea una variable importante para predecir ya que no se obtiene mucha correlacion.
- No podemos tener fechas en el entrenamiento del modelo asi que quitamos `reservation_status_date` ya que no creemos que sea muy influyente para la prediccion

In [ ]:
columns_to_drop = ['reservation_status_date', 'country', 'reservation_status']
df_hoteles_aux = df_hoteles.drop(columns=columns_to_drop)

 Ahora modificamos la columna de `arrival_date_month` para que contenga los numeros de cada mes en vez de su nombre, asi conseguimos incluir en el modelo esta variable y no tenemos que hacer un encoding agregando dimensionalidad

In [ ]:
MONTHS = {'January': 1, 
          'February': 2,
          'March': 3, 
          'April': 4, 
          'May': 5, 
          'June': 6, 
          'July': 7, 
          'August': 8, 
          'September': 9, 
          'October': 10, 
          'November': 11, 
          'December': 12
         }

def month_to_number(month):
    return MONTHS[month]

df_hoteles_aux['arrival_date_month'] = df_hoteles_aux['arrival_date_month'].apply(month_to_number)

Por ultimo encodeamos con la funcion `get_dummies` todas las variables categoricas para poder incluirlas en el entrenamiento de nuestro modelo

In [ ]:
columns_to_encode = [
                     'hotel', 
                     'meal', 
                     'reserved_room_type', 
                     'assigned_room_type',
                     'deposit_type',
                     'customer_type',
                     'distribution_channel',
                     'market_segment'
                    ]

df_hoteles_aux = pd.get_dummies(df_hoteles_aux, columns=columns_to_encode, dummy_na=False)

Separamos nuestro dataset en test y entrenamiento con un 20% y 80% respectivamente para evitar overfittear y tener resultados mas adecuados

In [ ]:
df_trabajo_x = df_hoteles_aux.drop(['is_canceled'], axis="columns" ,inplace=False)
df_trabajo_y = df_hoteles_aux['is_canceled']

X_train, X_test, Y_train, Y_test = train_test_split(df_trabajo_x, df_trabajo_y, stratify=df_trabajo_y, test_size=0.2, random_state=42)

# Optimización de hiperparámetros

## 1. Arbol de decision

Utilizamos ahora la tecnica de Grid Search para buscar los mejores hiperparametros para el modelo de arbol de decision, luego entrenamos el clasificador con los hiperparametros hallados

In [ ]:
params = {
    'criterion': ['gini'],
    'max_depth':[None],
    'max_features' : [0.7],
    'min_samples_split': [25], 
    'min_samples_leaf': [1]
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state= 42), params, scoring='roc_auc', n_jobs=-1, cv=5).fit(X_train, Y_train)
best_params = grid_search.best_params_
best_params

**1. ¿Cuántos folds utilizaron?**

    Se utilizaron 5 folds.

**2. ¿Qué métrica consideran adecuada para buscar los parámetros?**

    Se considero adecuada la metrica "roc auc"

In [ ]:
arbol = DecisionTreeClassifier(
    max_depth = best_params["max_depth"], 
    min_samples_leaf = best_params["min_samples_leaf"],
    max_features = best_params["max_features"],
    random_state = 42
)

arbol.fit(X_train, Y_train)

Realizamos la prediccion con el modelo que nos dio los mejores parametros con grid search

In [ ]:
y_pred2 = arbol.predict(X_test)
print(classification_report(Y_test, y_pred2))
print("ROC-AUC score:", round(metrics.roc_auc_score(Y_test, y_pred2), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(Y_test, y_pred2)

Para el plot del arbol resultante tuvimos que limitar la profunidad ya que tomaba muchisimo tiempo generarlo entero

In [ ]:
plt.figure(figsize=(10, 8), dpi=300)
graph = plot_tree(arbol, feature_names= grid_search.feature_names_in_, class_names=['not canceled','canceled'], max_depth=3)

## 2. Random forest

Ahora entrenamos con la division que hicimos del dataset a un modelo de random forest con nuevos hiperparametros

In [ ]:
rf_params = {
    'n_estimators': [150],  
    'max_features': [50],
    'max_depth': [13]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42),
                 rf_params,
                 cv = 5,
                 n_jobs=-1,
                 scoring = 'roc_auc').fit(X_train, Y_train)
best_params = grid_search.best_params_
best_params

In [ ]:
random_forest = RandomForestClassifier(
    n_estimators = best_params["n_estimators"], 
    max_features = best_params["max_features"],
    max_depth = best_params["max_depth"],
    random_state = 42
)

random_forest.fit(X_train, Y_train)

Lo evaluamos con todas las metricas que utilizamos para evaluar el arbol de decisiones anteriormente

In [ ]:
y_pred2 = random_forest.predict(X_test)
print(classification_report(Y_test, y_pred2))
print("ROC-AUC score:", round(metrics.roc_auc_score(Y_test, y_pred2), 4))

In [ ]:
ConfusionMatrixDisplay.from_predictions(Y_test, y_pred2)